Connecting to the Elasticsearch cluster

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient
from getpass import getpass

index = "devrel"

#Connect to the elastic cloud server
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY, # your username and password for connecting to elastic, found under Deplouments - Security
)

For NLP use cases we can import [models from the Hugging Face model hub](https://huggingface.co/elastic/distilbert-base-uncased-finetuned-conll03-english)

You can add them to the Elasticsearch cluster via docker.

https://www.elastic.co/guide/en/elasticsearch/client/eland/current/machine-learning.html#ml-nlp-pytorch-docker


In [ ]:
!docker pull docker.elastic.co/eland/eland

!docker run -it --rm elastic/eland \
    eland_import_hub_model \
      --cloud-id $ELASTIC_CLOUD_ID \
      --es-api-key $ELASTIC_API_KEY \
      --hub-model-id distilbert-base-uncased-finetuned-sst-2-english \
      --task-type text_classification \
      --start 

See all models successfuly imported into your instance (which are also available through the UI)

https://www.elastic.co/guide/en/elasticsearch/reference/current/get-trained-models.html?#ml-get-trained-models-request 

In [ ]:
models = MlClient.get_trained_models(es)
for model in models["trained_model_configs"]:
    print(model["model_id"])

You can test the models by running a query via the clients or (sometimes quicker and easier) through the Dev Console on Elastic

In [ ]:
model_id = "sentence-transformers__msmarco-minilm-l-12-v3"
models = MlClient.get_trained_models(es, model_id=model_id)
models.body

#Run a query againt the model - this is the format the query imput must be used in, you can later map your features into this format through an ingest pipeline
doc_test = {"text_field": "This is a very nice sentence"}

result = MlClient.infer_trained_model(es, model_id =model_id, docs = doc_test)
result["inference_results"]

## Pipelines to add LLM insights to your index

If you want to run your model against the entire index, you can build a pipeline to either run at the initial data ingestion, or to reindex your existing data with the new LLM insights.

You can build a pipeline for each model, or put them all together if you want multiple types of fields added at once.


In [ ]:
es.ingest.put_pipeline(
    id="sentiment_and_embeddings", 
    processors=[
    { #sentiment analysis example
      "inference": {
        "model_id": "distilbert-base-uncased-finetuned-sst-2-english",
        "target_field" : "sentiment",
        "field_map": {
          "Sentence": "text_field"
        }
      }
    },
    { #ELSER sparse vector example
      "inference": {
        "model_id": ".elser_model_1",
        "target_field": "ml",
        "field_map": {
          "Sentence": "text_field"
        },
        "inference_config": {
          "text_expansion": {
            "results_field": "tokens"
          }
        }
      }
    },
    { #embedding to dense vector example
      "inference": {
        "model_id": "sentence-transformers__msmarco-minilm-l-12-v3",
        "target_field" : "text_embedding",
        "field_map": {
          "Sentence": "text_field"
        }
      }
    }
  ]
)

For the new enriched index, we need to add a field to the mappings where the inference results will be stored. 
Here are some examples for different types of models:

In [ ]:
mappings = {
  "properties": {
    #dense vectors for embeddings
    "text_embedding.predicted_value": {
      "type": "dense_vector",
      "dims": 384,
      "index": True,
      "similarity": "cosine"
    },
    "Character": {
        "type": "text"
    },
    "Line_number": {
      "type": "long"
    },
    "Sentence": {
      "type": "text"
     },
     #sentiment results field
    "sentiment": {
      "properties": {
        "model_id": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
              }
          }
        },
        "predicted_value": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "prediction_probability": {
          "type": "float"
        }
      }
    }, #ELSER sparse vectors
    "ml.tokens": { 
      "type": "rank_features" 
    }
  }
}

In [ ]:
#Creating the new index with enriched data
es.indices.create(index=index, mappings=mappings)
es.reindex(body={
      "source": {
          "index": "index_name"},
      "dest": {"index": "index_name_new", "pipeline" : "sentiment"}
    }, wait_for_completion=False)

## Queries based on LLM fields

An example searching using the new sentiment field


In [ ]:
query={
    "match" : {
      "sentiment.predicted_value": "NEGATIVE"
    }
  }
response = es.search(index = "index_name_new",query=query, sort="sentiment.prediction_probability:desc")
print("The most negative sentences in the series:")
for hit in response["hits"]["hits"]:
    print(hit['_source']["Sentence"],  hit['_source']["sentiment"]["prediction_probability"] )

An example searching using the new embeddings field

In this case we also have to first convert the query text to the same embeddings space by also running this sentence against our inference model


In [ ]:
question = "what languages do you use"

question_embedded = MlClient.infer_trained_model(es, model_id =model_id, docs = question)
query_vector = question_embedded["inference_results"][0]["predicted_value"]

query = {
        "field": "text_embedding.predicted_value",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
        }
        
result = es.search(index = index, knn=query, source=["Sentence"])

for element in result["hits"]["hits"]:
    print("{}: {}, score {}".format(element["_source"]["Sentence"], element["_score"]))

When using ELSER, you don't need the extra step of embedding the question yourself, as this is done automatically. 

So the model takes in the string question directly:

In [ ]:
result = es.search(
    index=index, 
    size=5,
    query={
        "text_expansion": {
            "ml.tokens": {
                "model_id":".elser_model_1",
                "model_text":question
            }
        }
    }
)

for element in result["hits"]["hits"]:
    print("{}: {}, score {}".format(element["_source"]["Sentence"], element["_score"]))

You can also combine multiple types of search - like the ELSER semantic search with the sentiment field and some "classic" filters:

In [ ]:
result = es.search(
    index=index, 
    size=5,
    query={
        "bool": {
            "should": [{
                "text_expansion": {
                    "ml.tokens": {
                        "model_id":".elser_model_1",
                        "model_text":"brave"
                    }
                },
            }],
            "must":[
            {
                "match" : {
                    "sentiment.predicted_value": "NEGATIVE"
                }
            }],
            "must_not":[
                    {"term":{
                        "Sentence":"fear"
                 }}]
        }
    })

for element in result["hits"]["hits"]:
        print("{}: {}".format(element["_source"]["Character"], element["_source"]["Sentence"]))